In [ ]:
!pip install faiss-gpu
!pip install faiss-cpu
!pip install -U sentence-transformers

In [ ]:
import os
import faiss
import time
import urllib.request
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# 데이터 로드

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename = 'abcnews-date-text.csv')
df = pd.read_csv('abcnews-date-text.csv')
df.head()

In [ ]:
data = df['headline_text'].to_list()
data[:5]

In [ ]:
print('총 샘플의 개수 :', len(data))

# SBERT 임베딩

In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print('임베딩 된 벡터의 수 :', len(encoded_data))

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, 'abc_news')

In [ ]:
def search(query):
  t = time.time()
  query_vector = model.encode([query])
  k = 5
  top_k = index.search(query_vector, k)
  print(f'total time : {time.time() - t}')
  return [data[_id] for _id in top_k[1].tolist()[0]]

In [ ]:
query = str(input())
results = search(query)

print('results :')
for result in results:
  print('\t', result)